# Preparation

This part will help you get access to Gemini API and run your first prompting task.

For a detailed API walkthrough, see [this documentation page](https://ai.google.dev/tutorials/python_quickstart).


In [1]:
import sys
import os
import logging
import pathlib
import textwrap

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

## Setup your API key
Before you can use the Gemini API, you must first obtain an API key. If you don't already have one, create a key with one click in [Google AI Studio](https://makersuite.google.com/app/apikey). After obtaining the API key add it to your environment variables.

In [16]:
GOOGLE_API_KEY=os.getenv('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)

## List models

Now you're ready to call the Gemini API. Use list_models to see the available Gemini models:


In [18]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-pro
models/gemini-pro-vision


## Use this part to do your prompting

use the `gemini-pro` model.

In [10]:
model = genai.GenerativeModel('gemini-pro')
response = model.generate_content("What is HCI?")
to_markdown(response.text)

> **HCI** stands for **Human-Computer Interaction**.
> 
> It is the study of how people interact with computers and other related devices. It encompasses a wide range of topics, including:
> 
> * **User experience (UX)**: The overall experience that users have when interacting with a computer system. This includes factors such as the system's ease of use, usability, and aesthetics.
> * **User interface (UI)**: The part of a computer system that users interact with directly. This includes the graphical user interface (GUI), command line interface (CLI), and other forms of interaction.
> * **Accessibility:** The ability of users with disabilities to interact with computer systems. This includes providing alternative forms of input and output, such as screen readers and closed captioning.
> * **Ergonomics:** The study of how computer systems can be designed to minimize discomfort and injury for users. This includes factors such as the design of the keyboard and mouse, and the positioning of the computer screen.
> 
> HCI is a multidisciplinary field that draws on knowledge from psychology, computer science, design, and other disciplines. It is an important field because it can help to make computer systems more usable, accessible, and enjoyable for users.

# Writeup 0: Task and Strategy Description

Here, you should describe what task you are working on, and what workflow/pipeline you intend to replicate (from which crowdsourcing paper). As a reminder, you should [pick a crowdsourcing paper here](https://docs.google.com/spreadsheets/d/1qM66oZ5YcSjqifegoyNgl3LFMaCfFHgoPe8isgckbXA/edit?usp=sharing). The spreadsheet also points to example tasks in the crowdsourcing papers; However, you DON'T have to stick to the paper-provided input/output. Please feel free to come up with your own tasks as long as they seem suitable for the paper/pipeline you are replicating.


**EDIT THIS PART TO PROVIDE AN OVERVIEW OF YOUR ATTEMPTS**.

- **Task Description**: Create three metaphors for a given concept, so that we can explain the different aspects of crowdsourcing in a poetic way. A metaphor may look like:

    > In crowdsourcing, people are like bees; they work together to make honey.

    With the concept being “crowdsourcing”, the simile being “bees”, and the similar aspect being “work together.”
    
    This task is from paper [AI Chains: Transparent and Controllable Human-AI Interaction by Chaining Large Language Model Prompts](https://arxiv.org/pdf/2110.01691.pdf), Case 0 in Appendix B.3.
- **Example Input/output**: Write >=3 input-output pairs of your task. You should test your strategy on all the three examples.
    ```
    Input: Crowdsourcing
    Output: 
    1. crowdsourcing is like a team sport in that it brings people to achieve one goal.
    2. Crowdsourcing is like a vast, open-source library, with everyone contributing to the shelves of knowledge.
    3. Crowdsourcing was like a quilt, with each individual contributing a patch of fabric to create a larger piece of art.

    Input: Gratitude
    Output:
    1. gratitude is like a stream in that it’s a force that can carry you along.
    2. Gratitude is like a generous river that overflows with blessings.
    3. Gratitude is a feather in the cap of humility.

    Input: loss
    Output:
    1. loss is like a wing in that it’s something you never wanted to lose, and it can take you away.
    2. The loss was like a dark cloud of disappointment hovering over me.
    3. Loss is a heavy burden, weighing down the heart like a funeral shroud.
    ```
- **Workflow prompting strategy**: The pipeline helps further specify an abstract query into more specific aspects. Given an input concept, we will
    - First find three unique traits of the concept
    - Ask the model to write a different metaphor for each trait.
    
    For example:
    ```
    INPUT: gratitude
    Output: 
    TRAIT => OUTPUT: 
    Appreciation => Gratitude is a warm embrace, showing appreciation for all that is given.
    Generosity => Gratitude is like a generous river that overflows with blessings.
    ```
- **Crowdsourcing paper**: [Towards Large-Scale Collaborative Planning: Answering High-Level Search Queries Using Human Computation](https://ojs.aaai.org/index.php/AAAI/article/view/8092) -- where the pipelining strategy comes from.

## Baseline prompting

Use this section to do a one-step prompting for your selected task.

In [12]:
# Write your code here.
def baseline_prompt(concept, model='gemini-pro'):
    prompt = f"""The following is a list of three metaphors on '{concept}'.
Concept: {concept}
Metaphors: 1."""
    
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt)
    print(f"INPUT: {concept}")
    print(f"OUTPUT: {response.text}")
    print()
    
baseline_prompt("crowdsourcing")

baseline_prompt("gratitude")

baseline_prompt("loss")

INPUT: crowdsourcing
OUTPUT: 1. A collective brain: Crowdsourcing is like tapping into the collective intelligence of a vast pool of individuals, harnessing their knowledge, skills, and perspectives to solve problems or create innovative solutions.
2. A jigsaw puzzle: Crowdsourcing involves breaking down a complex task or problem into smaller pieces and assigning them to individuals or teams. Each piece represents a different perspective or expertise, and when assembled together, they form a complete solution.
3. A global marketplace: Crowdsourcing opens up a marketplace where individuals and organizations can connect and collaborate, exchanging ideas, knowledge, and resources to achieve shared goals or address common challenges.

INPUT: gratitude
OUTPUT: It is not possible to generate metaphors without a context. Please provide a context or specific example so that I can provide an appropriate metaphor.

INPUT: loss
OUTPUT: 1. A gaping hole in the fabric of life
2. A dark void that co

## Crowdsourcing-Pipeline-inspired prompting

Use this section to your selected pipeline prompting for your task.

In [14]:
# Write your code here.
from functools import partial
import re

def pipeline_prompt(concept, model='gemini-pro'):
    prompt_concept_to_trait = f"""The following three phrases describes three different and unique traits of the given concept.
Concept: {concept}
Unique traits: 1."""
   
    model = genai.GenerativeModel('gemini-pro')
    response = model.generate_content(prompt_concept_to_trait)
    traits = response.text
    traits = [t.strip() for t in re.split(r'\d+\.', traits) if t.strip()]
    
    metaphors = []
    for trait in traits:
        prompt_trait_to_metaphor = f"""Given the concept and its unique trait, write a metaphor that conveys the corresponding trait.
Concept: {concept}
trait: {trait}
metaphor:"""
        response = model.generate_content(prompt_trait_to_metaphor)
        metaphors.append(response.text.strip())
    metaphor = '\n'.join(metaphors)
    print(f"INPUT: {concept}")
    print(f"TRAITS: \n{traits}")
    print(f"OUTPUT: \n{metaphor}")
    print()

pipeline_prompt("crowdsourcing")

pipeline_prompt("gratitude")

pipeline_prompt("loss")

INPUT: crowdsourcing
TRAITS: 
['Distributed problem-solving']
OUTPUT: 
Crowdsourcing: A global assembly line where minds collaborate like cogs, powering solutions to complex puzzles.

INPUT: gratitude
TRAITS: 
["* **Appreciation of the present moment:** Gratitude is about acknowledging and appreciating the good things in our lives, no matter how small. It's about being present and taking the time to notice and savor the things that make us happy and grateful.\n* **Acknowledgment of others:** Gratitude is about recognizing the role that others play in our lives. It's about being thankful for the people who support us, help us, and make our lives better. It's about expressing our appreciation for the things that others do for us, both big and small.\n* **Awareness of the good in the world:** Gratitude is about recognizing the good that exists in the world, even when things are tough. It's about being aware of the beauty and the joy that surrounds us, even when it's easy to focus on the n

# Writeup 1: Report & Reflection

Fill in the following three sections by reflecting on your results.

## Report the result

For all the inputs you tried, summarized the input, baseline output, pipeline output, which output you like and why. To answer "why", you should first think of some criteria you want to use for evaluating the output:

### Important criteria:
1. Diversity: The output should be diverse.
2. Clarity: The output should be descriptive and clearly explains why a metaphor is used.

### Inputs and outputs:

- **Input**: loss
- **Baseline output**: (diversity: 5/5, clarity: 1/5)
    - His heart was an empty void. 
    - The grief was a heavy weight on his shoulders. 
    - He felt like he was walking through a dark tunnel.

- **Pipeline output**: (diversity: 5/5, clarity: 1/5)
    - The world was a barren desert, devoid of hope and full of desolation.
    - Disappointment settled over the room like a thick fog, obscuring the joy that had been there before.
    - The grief of loss was like a balloon set free, slowly fading away in the distance.

- **Which output you like and why**:
    I like the pipeline output, because it's much more clear why a simile is picked.

--- 

- **Input**:
- **Baseline output**:
- **Pipeline output**:
- **Which output you like and why**:


--- 

...

## Reflect on prompting effectiveness

Write some paragraphs to describe: Did you find the pipeline prompting workflow effective? Why or Why not?

## Envision possible improvements

Write some paragraphs to describe: What are some possible ways to further improve the pipeline or prompt design?